In [1]:
!pwd

/Users/ashish1610dhiman/data_projects/bestbuy/notebooks/ashish_validation_train


In [2]:
import sys
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

sys.version

sys.path.append("../../")

from src.ad_hmm import sku_predict
from src.ad_stl_model import stl_model
from src.ad_stl_prophet import stl_model
from src.utils import *
import pickle

Importing plotly failed. Interactive plots will not work.


In [3]:
%load_ext autoreload 
%autoreload 2

### Get STL/Prophet SKUs

In [4]:
with open('../../results/final_sku_model_map.pkl', 'rb') as handle:
    sku_period_dict1 = pickle.load(handle)

In [5]:
sku_period_dict1.keys()

dict_keys(['rmse_du_15', 'rmse_du_30', 'rmse_du_45', 'rmse_du_60', 'rmse_du_90', 'rmse_du_MSTL', 'rmse_du_STL', 'rmse_du_holt', 'rmse_du_max', 'rmse_du_null', 'rmse_du_prophet'])

In [6]:
stl_keys = ['rmse_du_MSTL', 'rmse_du_STL', 'rmse_du_prophet']

In [9]:
import itertools
stl_skus = [sku_period_dict1[sub_model] for sub_model in stl_keys]
stl_skus = list(itertools.chain.from_iterable(stl_skus))

len(stl_skus),len(set(stl_skus))

(141, 127)

In [10]:
stl_skus = list(set(stl_skus))

### Prepare Train data

In [20]:
original_train = pd.read_csv("../../data/train_validation_marker.csv").drop_duplicates()
original_train = original_train.loc[original_train.Encoded_SKU_ID.isin(stl_skus)]
original_train["SALES_DATE"] = pd.to_datetime(original_train["SALES_DATE"])
original_train["validation_clean"] = True
original_train["validation"] = False

In [21]:
original_validation = pd.read_excel("../../data/Validation_Data.xlsx").drop_duplicates()
original_validation = original_validation.loc[original_validation.Encoded_SKU_ID.isin(stl_skus)]
original_validation["SALES_DATE"] = pd.to_datetime(original_validation["SALES_DATE"])
original_validation["validation_clean"] = True
original_validation["validation"] = True

mute Daily units before appending 

In [22]:
original_validation["DAILY_UNITS"] = np.nan

In [23]:
common_cols = set(original_train.columns).intersection(original_validation.columns)
original_train1 = pd.concat([original_train[list(common_cols)],original_validation[list(common_cols)]])
print (original_train.shape,original_validation.shape,original_train1.shape)

(177736, 15) (889, 14) (178625, 13)


In [24]:
original_train1["if_promo"] = (~original_train1["PROMO_PRICE"].isin(["?"])).astype("int")
original_train1["if_competitor"] = (~original_train1["COMPETITOR_PRICE"].isin(["?"])).astype("int")

original_train1["min_sales_price"] = pd.to_numeric(np.where(original_train1.if_promo>0, original_train1.PROMO_PRICE,\
                                                       original_train1.RETAIL_PRICE))

original_train1["comp_price_clean"] = pd.to_numeric(np.where(original_train1.if_competitor>0,original_train1.COMPETITOR_PRICE, np.nan))
original_train1["price_delta"] = np.where(original_train1.if_competitor>0, original_train1.min_sales_price - original_train1.comp_price_clean,\
                                   -original_train1.min_sales_price)

In [25]:
original_train1 = pd.get_dummies(original_train1,columns = ["Inventory"], drop_first=False)

In [26]:
common_cols

{'CATEGORY_NAME',
 'CLASS_NAME',
 'COMPETITOR_PRICE',
 'DAILY_UNITS',
 'Encoded_SKU_ID',
 'Inventory',
 'ML_NAME',
 'PROMO_PRICE',
 'RETAIL_PRICE',
 'SALES_DATE',
 'SUBCLASS_NAME',
 'validation',
 'validation_clean'}

In [27]:
original_train1.columns

Index(['validation', 'PROMO_PRICE', 'CATEGORY_NAME', 'CLASS_NAME',
       'DAILY_UNITS', 'COMPETITOR_PRICE', 'RETAIL_PRICE', 'ML_NAME',
       'validation_clean', 'SALES_DATE', 'Encoded_SKU_ID', 'SUBCLASS_NAME',
       'if_promo', 'if_competitor', 'min_sales_price', 'comp_price_clean',
       'price_delta', 'Inventory_Constrained', 'Inventory_Fully-Stocked',
       'Inventory_Moderate', 'Inventory_Out-of-Stock'],
      dtype='object')

#### Pivot data

In [28]:
original_train1[original_train1.duplicated(keep=False)]

,validation,PROMO_PRICE,CATEGORY_NAME,CLASS_NAME,DAILY_UNITS,COMPETITOR_PRICE,RETAIL_PRICE,ML_NAME,validation_clean,SALES_DATE,...,SUBCLASS_NAME,if_promo,if_competitor,min_sales_price,comp_price_clean,price_delta,Inventory_Constrained,Inventory_Fully-Stocked,Inventory_Moderate,Inventory_Out-of-Stock


In [31]:
original_train_pivot = original_train1.pivot(index='SALES_DATE', columns='Encoded_SKU_ID',\
                   values=['DAILY_UNITS',"if_promo","if_competitor","min_sales_price","price_delta",\
                          "RETAIL_PRICE",'Inventory_Constrained', 'Inventory_Fully-Stocked',
       'Inventory_Moderate', 'Inventory_Out-of-Stock',"validation","validation_clean"])
original_train_pivot.columns = original_train_pivot.columns.swaplevel(0, 1)

### Test code for 1 SKU

In [32]:
sku_id = stl_skus[0]
train_test_sku = original_train_pivot.loc[:,(sku_id,)].copy()
train_test_sku["ad_time"] = list(range(1,original_train_pivot.shape[0]+1))

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2833426789.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  train_test_sku = original_train_pivot.loc[:,(sku_id,)].copy()


In [33]:
exog_cols = ["if_promo","if_competitor","min_sales_price","price_delta","RETAIL_PRICE",\
            'Inventory_Constrained', 'Inventory_Fully-Stocked',
       'Inventory_Moderate', 'Inventory_Out-of-Stock']
stl_model1 = stl_model(sku_id = sku_id,\
                      train_test_sku = train_test_sku,\
                      n_lags_y = 30, \
                      n_lags_exog = 14, \
                      pred_cols = exog_cols)

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
m_dict, y_dict = stl_model1.fit_models();

23:04:40 - cmdstanpy - INFO - Chain [1] start processing
23:04:41 - cmdstanpy - INFO - Chain [1] done processing


(518, 'none', 'prophet'), Train RMSE = 1.426

(518, 'STL_resid', 'auto_reg'), Train RMSE = 1.087
(518, 'STL_resid', 'xgb_340'), Train RMSE = 0.8887
(518, 'STL_resid', 'xgb_1000'), Train RMSE = 0.853

(518, 'STL_trend', 'auto_reg'), Train RMSE = 2.727e-05
(518, 'STL_trend', 'xgb_340'), Train RMSE = 0.1175
(518, 'STL_trend', 'xgb_1000'), Train RMSE = 0.1175

(518, 'STL_seasonal', 'auto_reg'), Train RMSE = 1.1
(518, 'STL_seasonal', 'xgb_340'), Train RMSE = 0.6017
(518, 'STL_seasonal', 'xgb_1000'), Train RMSE = 0.6017

(518, 'MSTL_resid', 'auto_reg'), Train RMSE = 0.864
(518, 'MSTL_resid', 'xgb_340'), Train RMSE = 1.004
(518, 'MSTL_resid', 'xgb_1000'), Train RMSE = 1.001

(518, 'MSTL_trend', 'auto_reg'), Train RMSE = 4.943e-05
(518, 'MSTL_trend', 'xgb_340'), Train RMSE = 0.2366
(518, 'MSTL_trend', 'xgb_1000'), Train RMSE = 0.2366

(518, 'MSTL_seasonal_7', 'auto_reg'), Train RMSE = 0.05155
(518, 'MSTL_seasonal_7', 'xgb_340'), Train RMSE = 0.5558
(518, 'MSTL_seasonal_7', 'xgb_1000'), Train R

In [40]:
min_models, y_pred = stl_model1.get_result();

In [41]:
min_models

,component,model_name,model,train_rmse,rnk
0,none,prophet,<prophet.forecaster.Prophet object at 0x15ba58...,1.426143,1.0
3,STL_resid,xgb_1000,ForecastingCascade(estimator=XGBRegressor(base...,0.852983,1.0
4,STL_trend,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.000027,1.0
9,STL_seasonal,xgb_1000,ForecastingCascade(estimator=XGBRegressor(base...,0.601684,1.0
10,MSTL_resid,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.86404,1.0
13,MSTL_trend,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.000049,1.0
16,MSTL_seasonal_7,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.051552,1.0
19,MSTL_seasonal_30,auto_reg,<statsmodels.tsa.ar_model.AutoRegResultsWrappe...,0.102354,1.0
24,MSTL_seasonal_365,xgb_1000,ForecastingCascade(estimator=XGBRegressor(base...,0.86868,1.0


In [43]:
y_pred

,STL_prediction,MSTL_prediction,prophet_prediction,Encoded_SKU_ID,actual
SALES_DATE,,,,,
2022-08-01,-0.053428,0.877414,0.634117,518,NaN
2022-08-02,0.433740,-0.119518,0.777299,518,NaN
2022-08-03,0.808414,2.748224,0.836469,518,NaN
2022-08-04,0.133989,-0.578705,0.804469,518,NaN
2022-08-05,-0.018746,-0.415380,0.718778,518,NaN
2022-08-06,1.991127,-0.452726,1.883775,518,NaN
2022-08-07,1.937726,-0.664471,1.585911,518,NaN


### Run for all SKUs

In [44]:
from joblib import Parallel, delayed
from IPython.utils import io
from tqdm import tqdm

In [45]:
def fit_one_sku(train_test_pivot1,sku_id1, n_lags_y1, n_lags_exog1, exog_cols1):
    train_test_sku1 = train_test_pivot1.loc[:,(sku_id1,)].copy()
    train_test_sku1["ad_time"] = list(range(1,train_test_sku1.shape[0]+1))
    stl_model1 = stl_model(sku_id = sku_id1,\
                      train_test_sku = train_test_sku1,\
                      n_lags_y = n_lags_y1, \
                      n_lags_exog = n_lags_exog1, \
                      pred_cols = exog_cols1)
    try:
        with io.capture_output() as captured:
            m_dict, y_dict = stl_model1.fit_models();
            min_models, y_pred = stl_model1.get_result();
        print (f"Successful for {sku_id1}")
        return (min_models, y_pred)
    except:
        print (f"Error for {sku_id1}")

In [46]:
len(stl_skus)

127

In [49]:
all_preds = Parallel(n_jobs=4)(delayed(fit_one_sku)(train_test_pivot1 = original_train_pivot,\
                                                    sku_id1 = sku_id_i,\
                                                    n_lags_y1 = 30,\
                                                    n_lags_exog1 = 14,\
                                                    exog_cols1 = exog_cols) \
                               for sku_id_i in tqdm(stl_skus))


  0%|                                                                                                                                                  | 0/127 [00:00<?, ?it/s]Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth 

 35%|██████████████████████████████████████████████▊                                                                                        | 44/127 [36:04<1:33:45, 67.77s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 38%|███████████████████████████████████████████████████                                                                                    | 48/127 [41:30<1:34:42, 71.93s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.

 69%|███████████████████████████████████████████████████████████████████████████████████████████▍                                        | 88/127 [1:54:16<1:18:18, 120.48s/it]/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipyke

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_53609/2183440621.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.


In [50]:
len(all_preds)

127

In [51]:
import pickle
with open('../../data/stl_prophet_final_results_127sku.pkl', 'wb') as f:
    pickle.dump(all_preds, f)